![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Create SQL managed instance - Azure Arc on an existing Azure Arc Data Controller
 
This notebook walks through the process of creating a <a href="https://docs.microsoft.com/azure/sql-database/sql-database-managed-instance">SQL managed instance - Azure Arc</a> on an existing Azure Arc Data Controller.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target Azure Arc Data Controller already created.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|Azure Data CLI (azdata) | Command-line tool for installing and managing resources in an Azure Arc cluster |[Installation](https://docs.microsoft.com/sql/azdata/install/deploy-install-azdata) |

### **Setup and Check Prerequisites**

In [ ]:
import sys,os,json,subprocess
def run_command():
    print("Executing: " + cmd)
    output = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True )
    if output.returncode != 0:
        print(f'Command: {cmd} failed \n')
        print(f'\t>>>Error output: {output.stderr.decode("utf-8")}\n')
        sys.exit(f'exit code: {output.returncode}\n')
    print(f'Successfully executed: {cmd}')
    print(f'\t>>>Output: {output.stdout.decode("utf-8")}\n')
    return output.stdout.decode("utf-8")
cmd = 'azdata --version'
out = run_command()


### **Set variables**

#### 

Generated by Azure Data Studio using the values collected in the 'Deploy Azure SQL managed instance - Azure Arc' wizard

### **Creating the SQL managed instance - Azure Arc instance**

In [ ]:
# Login to the data controller.
#
os.environ["AZDATA_PASSWORD"] = os.environ["AZDATA_NB_VAR_CONTROLLER_PASSWORD"]
os.environ["KUBECONFIG"] = controller_kubeconfig
os.environ["KUBECTL_CONTEXT"] = controller_kubectl_context
cmd = f'azdata login -e {controller_endpoint} -u {controller_username}'
out=run_command()

In [ ]:
print (f'Creating the SQL managed instance - Azure Arc instance')

cores_request_option = f' -cr "{sql_cores_request}"' if sql_cores_request else ""
cores_limit_option = f' -cl "{sql_cores_limit}"' if sql_cores_limit else ""
memory_request_option = f' -mr "{sql_memory_request}Gi"' if sql_memory_request else ""
memory_limit_option = f' -ml "{sql_memory_limit}Gi"' if sql_memory_limit else ""

os.environ["AZDATA_USERNAME"] = sql_username
os.environ["AZDATA_PASSWORD"] = os.environ["AZDATA_NB_VAR_SQL_PASSWORD"]
cmd = f'azdata arc sql mi create -n {sql_instance_name} -scd {sql_storage_class_data} -scl {sql_storage_class_logs}{cores_request_option}{cores_limit_option}{memory_request_option}{memory_limit_option}'
out=run_command()